<a href="https://colab.research.google.com/github/harshalDharpure/HateFul-Hindi-Memes-Detection/blob/main/Caption_and_Data_Loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install beautifulsoup4 requests


In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def download_memes(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch content from {url}. Status code: {response.status_code}")
        return

    soup = BeautifulSoup(response.content, 'html.parser')
    image_tags = soup.find_all('img')

    for idx, img_tag in enumerate(image_tags):
        img_url = img_tag['src']
        img_url = urljoin(url, img_url)
        try:
            img_response = requests.get(img_url, stream=True)
            img_extension = img_url.split('.')[-1]

            # Save the image to a file
            file_name = f'meme_{idx}.{img_extension}'
            with open(file_name, 'wb') as img_file:
                for chunk in img_response.iter_content(chunk_size=8192):
                    img_file.write(chunk)

            print(f"Downloaded {file_name} successfully.")
        except requests.exceptions.RequestException as e:
            print(f"Failed to download image from {img_url}: {str(e)}")

if __name__ == "__main__":
    # Replace this with the URL containing the memes you want to download
    meme_url = "https://imgflip.com/tag/hot+chick"
    download_memes(meme_url)


In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def download_memes(url):
    # Create a 'data' directory if it doesn't exist
    if not os.path.exists('data'):
        os.makedirs('data')

    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch content from {url}. Status code: {response.status_code}")
        return

    soup = BeautifulSoup(response.content, 'html.parser')
    image_tags = soup.find_all('img')

    for idx, img_tag in enumerate(image_tags):
        img_url = img_tag['src']
        img_url = urljoin(url, img_url)
        try:
            img_response = requests.get(img_url, stream=True)
            img_extension = img_url.split('.')[-1]

            # Save the image to a file inside the 'data' folder
            file_name = f'data/meme_{idx}.{img_extension}'
            with open(file_name, 'wb') as img_file:
                for chunk in img_response.iter_content(chunk_size=8192):
                    img_file.write(chunk)

            print(f"Downloaded {file_name} successfully.")
        except requests.exceptions.RequestException as e:
            print(f"Failed to download image from {img_url}: {str(e)}")

if __name__ == "__main__":
    # Replace this with the URL containing the memes you want to download
    meme_url = "https://imgflip.com/tag/hot+chick"
    download_memes(meme_url)


In [ ]:
pip install easyocr

In [ ]:
import os
import csv
import easyocr

# Function to extract captions using EasyOCR
def extract_caption_using_easyocr(image_path):
    # Initialize EasyOCR reader
    reader = easyocr.Reader(['en'])

    try:
        # Extract text from the image
        result = reader.readtext(image_path, detail=0)
        caption = ' '.join(result)
    except Exception as e:
        print(f"Failed to extract caption from {image_path}: {str(e)}")
        caption = "Caption extraction failed."

    return caption

# Rest of the code remains unchanged

def create_tsv_with_easyocr(directory):
    rows = []
    for filename in os.listdir(directory):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(directory, filename)
            caption = extract_caption_using_easyocr(image_path)
            categories = categorize_meme(caption)
            rows.append([filename] + list(categories.values()) + [caption])

    # Write to TSV file
    with open('memes_data_with_easyocr.tsv', 'w', newline='') as tsvfile:
        writer = csv.writer(tsvfile, delimiter='\t')
        writer.writerow(["file_name", "misogynous", "shaming", "stereotype", "objectification", "violence", "Text_Transcription"])
        writer.writerows(rows)

if __name__ == "__main__":
    # Replace with the directory containing your meme images
    meme_directory = "/content/data"
    create_tsv_with_easyocr(meme_directory)
